In [ ]:
%run "P:\Experiments\006 RPE Image Analysis (Christin)\Functions.py" 

In [ ]:
# Define Mouse ID's, Mutations, and Tissue Types

mouse_num = ['1256', '1257', '1258', '1259', '1260', '1494', '1498', '1499', 
             '1500', '1501', '1502']
mut_types = ['WT', 'Het', 'KO']
age_type = ['P14', 'P30']
steps = 1

# Define smoothing kernel size and other stuff for later

smooth_size = 3
selem = disk(3)
Bc = np.ones((9,9))

In [ ]:
# Set working directory
%cd "P:\Experiments\006 RPE Image Analysis (Christin)\2019-06-25 CLC2 RPE Morphology\Images"

# Read in data from .tif
AF_647 = skimage.io.ImageCollection('*647.tif', load_func=imread_rgb)

ZO1 = [image for image in AF_647]

filenames = glob.glob('*647.tif')
filenames.sort(key=natural_keys) # Note the natural sorting definition so that 
                                 # filenames are in line with ImageCollection Images

In [ ]:
# Create factor lists to be used in dataframes later
mut = factor_maker(mut_types, filenames, steps)
mice = factor_maker(mouse_num, filenames, steps)

In [ ]:
%cd "P:\Experiments\006 RPE Image Analysis (Christin)\2019-06-25 CLC2 RPE Morphology\Results\Flat Mount Area"

# Local thresholding is utilized as flatmounts are rarely so flat
# as to be entirely within a single focal plane, which results in
# variable tissue stain intensity within the images. Binary dilation
# and fill holes are used to fill gaps. Lastly, the flat mount area
# is extracted by appending the largest non-background label.

fm_area_image = []
fm_area_value = []

for n, image in tqdm(enumerate(ZO1)):
    thresh = skimage.filters.threshold_local(image, 7999)
    binary = image > thresh
    binary_dil = skimage.morphology.binary_dilation(binary, disk(12))
    binary_closed = mh.close_holes(binary_dil)
    binary_label, n_mc = mh.label(binary_closed)
    
    label_areas = mh.labeled.labeled_size(binary_label)
    label_areas_noback = label_areas[1:]
    fm_area_image.append(binary_label == np.argsort(label_areas_noback)[-1] + 1)
    fm_area_value.append(np.sort(label_areas_noback)[-1] + 1)
    
    # Save flat mount area numpy arrays so they can be loaded later.
    np.save("Flat Mount Area Array %s %s" % (mice[n], mut[n]), 
            binary_label == np.argsort(label_areas_noback)[-1] + 1)
    
    plt.figure(figsize = (15, 15))
    
    plt.imshow(fm_area_image[n], cmap=rmap)
    plt.axis('off')

    plt.savefig("Flat Mount Area %s %s" % (mut[n], mice[n]), dpi=150)
    plt.close()   

In [ ]:
%cd "P:\Experiments\006 RPE Image Analysis (Christin)\2019-06-25 CLC2 RPE Morphology\Results"

cells = []

for n, image in enumerate(ZO1):
    neurites = enhance_neurites(image, sigma=2)
    thresh = skimage.filters.threshold_local(neurites, 99)
    binary = neurites > thresh
    # binary = skimage.filters.apply_hysteresis_threshold(test, 0.04, 0.25)
    skel = skimage.morphology.skeletonize(binary)
    closed = skimage.morphology.binary_closing(skel, selem = square(3))
    dil = skimage.morphology.binary_dilation(closed, selem = square(3))
    flat_dil = dil*fm_area_image[n]
    
    binary_label, n_cell1 = mh.label(skimage.util.invert(flat_dil))
    pre_cell_sizes = mh.labeled.labeled_size(binary_label)
    pre_multi_cell = np.where(pre_cell_sizes > 20000)
    pre_small_cell = np.where(pre_cell_sizes < 20)
    pre_binary2 = mh.labeled.remove_regions(binary_label, pre_multi_cell)
    pre_binary3 = mh.labeled.remove_regions(pre_binary2, pre_small_cell)
    
    binary_closed = mh.close_holes(pre_binary3)
    binary_closed_label, n_cell2 = mh.label(binary_closed)
    cell_sizes = mh.labeled.labeled_size(binary_closed_label)
    multi_cell = np.where(cell_sizes > 1500)
    small_cell = np.where(cell_sizes < 150)
    binary2 = mh.labeled.remove_regions(binary_closed_label, multi_cell)
    binary3 = mh.labeled.remove_regions(binary2, small_cell)

    cleared = skimage.segmentation.clear_border(binary3)
    cleared2, n_left = mh.labeled.relabel(cleared)
    
    cells.append(cleared2)
    
    # Save segmented numpy arrays so they can be loaded later.
    np.save("Cell Outline Array %s %s" % (mice[n], mut[n]), cleared2)
    
    plt.figure(figsize = (30, 30))
    plt.imshow(cleared2, cmap=rmap, interpolation='nearest')
    plt.axis('off')

    plt.savefig("Cell Outline %s %s" % (mice[n], mut[n]), dpi=300)
    plt.close()